In [11]:
import cv2
import time

# Phone camera configuration (MODIFY THESE)
PHONE_IP = "192.168.1.2"  # Replace with your phone's IP address
PORT = "8080"               # Default port for IP Webcam
URL = f"http://192.168.1.2:8080/video"

# Initialize phone camera stream
cap = cv2.VideoCapture(URL)

# Allow time for camera stream to initialize
time.sleep(2)

if not cap.isOpened():
    print("Failed to connect to phone camera")
    print("Troubleshooting:")
    print("1. Ensure both devices are on same network")
    print("2. Verify IP address and port in the script")
    print("3. Check if camera server is running on phone")
    exit()

# Read first frame
ret, frame = cap.read()
if not ret:
    print("Failed to get frame from phone camera")
    cap.release()
    exit()

# Select ROI
bbox = cv2.selectROI("Select Object", frame, False, False)
cv2.destroyWindow("Select Object")

if bbox == (0, 0, 0, 0):
    print("No object selected")
    cap.release()
    exit()

# Initialize tracker
tracker = cv2.legacy.TrackerCSRT_create()
tracker.init(frame, bbox)

# Tracking loop
while True:
    ret, frame = cap.read()
    if not ret:
        print("Connection lost with phone camera")
        break

    # Update tracker
    success, bbox = tracker.update(frame)

    # Draw bounding box
    if success:
        x, y, w, h = [int(v) for v in bbox]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        status = "Tracking"
    else:
        status = "Tracking failed"

    # Display status
    cv2.putText(frame, status, (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
    cv2.imshow("Phone Camera Tracking", frame)

    # Exit on ESC or 'q'
    key = cv2.waitKey(1)
    if key in [27, ord('q')]:
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()